In [1]:
%load_ext dotenv
%dotenv

In [2]:
import logging
import os
from solana.rpc.async_api import AsyncClient
from zeta_py.types import Network, Asset, Side, OrderOptions, OrderType
from solana.rpc.commitment import Confirmed
from solana.rpc.types import TxOpts
from zeta_py import utils
from zeta_py.client import Client
import anchorpy
from datetime import datetime, timedelta
import re
from solana.rpc.websocket_api import connect

logger = logging.getLogger("zeta_py")
logging.basicConfig(level=logging.DEBUG)

endpoint = os.getenv("ENDPOINT", "https://api.mainnet-beta.solana.com")
print(endpoint.split(".com")[0])
conn = AsyncClient(endpoint, commitment=Confirmed, blockhash_cache=False)

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet at ~/.config/solana/id.json
# print(wallet.public_key)

https://api.mainnet-beta.solana


In [3]:
tx_opts = TxOpts({"preflight_commitment": Confirmed, "skip_confirmation": False})

client = await Client.load(Network.MAINNET, conn, wallet, [Asset.SOL], subscribe=False, tx_opts=tx_opts)

INFO:zeta_py.accounts.Account:Loaded account: State
INFO:zeta_py.accounts.Account:Loaded account: Pricing
INFO:zeta_py.accounts.Account:Loaded account: Clock
INFO:zeta_py.accounts.Account:Subscribed to Clock
INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccount


In [4]:
commitment = Confirmed
ws_endpoint = "wss://api.mainnet-beta.solana.com"

# Subscriptions

## Accounts

### Orderbooks

In [ ]:
address = client.exchange.markets[Asset.SOL]._serum_market.state.bids()
async with connect(ws_endpoint) as ws:
  await ws.account_subscribe(
      address,
      commitment=commitment,
      encoding="base64+zstd",
  )
  first_resp = await ws.recv()
  subscription_id = first_resp[0].result
  async for msg in ws:
      account = client.exchange.markets[Asset.SOL]._serum_market._parse_bids_or_asks(msg[0].result.value.data)
      print(account._get_l2(5))

## Transactions

### Trades

In [ ]:
from anchorpy import EventParser
from solders.rpc.config import RpcTransactionLogsFilterMentions

address = client.exchange.markets[Asset.SOL]._serum_market.state.bids()
async with connect(ws_endpoint) as ws:
    await ws.logs_subscribe(
        commitment=commitment,
        filter_=RpcTransactionLogsFilterMentions(client.exchange.program_id)
    )
    first_resp = await ws.recv()
    subscription_id = first_resp[0].result
    async for msg in ws:
        logs = msg[0].result.value.logs
        parser = EventParser(client.exchange.program_id, client.exchange.program.coder)
        parsed = []
        parser.parse_logs(logs, lambda evt: parsed.append(evt))
        for event in parsed:
            if event.name == "ApplyFundingEvent":
                pass
            elif event.name == "PlaceOrderEvent":
                # if event.data.margin_account == client.margin_account.address:
                #     print(event)
                pass
            elif event.name == "OrderCompleteEvent":
                # if event.data.margin_account == client.margin_account.address:
                #     print(event)
                pass
            elif event.name == "TradeEventV3":
                if event.data.margin_account == client.margin_account.address:
                    print(event)
            elif event.name == "LiquidationEvent":
                # if event.data.liquidatee == client.marg
                pass
            else:
                print(event)